## Feb 16 2024

Topics
* Volumes vs Surfaces [pial, white, inflated] (freeview)
* fsnative vs fsaverage (T1w -> MNI)
* surface mesh vs surface data (cells below)
* plot sub 29 NC vals as overlay onto fsaverage inflated left hemisphere (freeview)
* configure -> 90%, explore
* mri_surf2surf conversion

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import nilearn
import nibabel as nib
import nilearn.surface 

from nilearn.image import load_img, mean_img
from nilearn import surface, plotting
from pathlib import Path

sub_num = '29'
vis_dir = Path(f'E:/fmri_processing/results/visualization/sub-{sub_num}')
anat_dir = Path(f'E:/fmri_processing/results/derivatives_TC2See/fmriprep/sub-{sub_num}/anat')
# surf_dir = Path(f'E:/fmri_processing/results/derivatives_TC2See_new/sourcedata/freesurfer/sub-{sub_num}/surf')

### Volumes vs Surfaces (freeview)

### `fsnative` vs `fsaverage` 

In volume-based MRI, we have functional data and the anatomical data. The anatomical data is a high-resolution scan of a specific participant's brain. This is called the T1(w) image. The MNI template is an average of > 100 brains that we map to so that results from multiple subjects are alignable. 

In the surface world, using Freesurfer, making surfaces that are subject-specific, this is called `fsnative` space. So a specific subject's T1w VOLUME is converted into a surface that we say is in `fsnative` SPACE. 

Going from T1w to MNI (average template) in terms of surfaces is going from `fsnative` -> `fsaverage` (`fsaverage` brain is based on MNI template).

### Surfaces: mesh vs overlays

A surface is defined by two arrays:
* vertex array of coordinates
* list of triangles (pointing to triplets of vertices in the vertex array)

Can get best understanding via playing around with examples...

### Nilearn Surfaces

* `nilearn.surface.load_surf_mesh`
* `nilearn.surface.load_surf_data`

The function `load_surf_mesh` loads the surface data that has (a) list of vertex locations and (b) the triangle ("faces") triplets that say which three vertices make up a specific triangle in the mesh.

The function `load_surf_data` loads a value of interest to be plotted on each of the vertices. The triangles and structure are already set up to give the 'shape' of the brain, but the colours to be plotted on there are called 'overlays' and you lay the values over the mesh. 

The noise ceiling values are numerical values that go on top of each vertex point, so this is an 'overlay'.

In [ ]:
import nilearn
from nilearn import datasets, surface
fsaverage = datasets.fetch_surf_fsaverage(mesh='fsaverage')

In [ ]:
fsaverage_infl_left = surface.load_surf_mesh(fsaverage.infl_left)
print(len(fsaverage_infl_left))
vertices, triangles = fsaverage_infl_left
print(f"{vertices.shape = }")
print(f"{triangles.shape = }")

## Overlays

If (vertices, triangles) define a surface mesh, then when we plot, we want to put a single value on each vertex in the mesh. This means an overlay should have the same number of vertices as the vertex list of the mesh and a single column value to plot. That's what the noise ceiling surface values are. 

But first, how to go from a volume of noise ceilings to a surface?

* Load the NC volumes previously calculated
* Load a surface file that `fMRIprep` calculates and has saved (we'll use `pial_lh`)
* Use the `surface.vol_to_surf` function to derive an overlay
* Use `nibabel` to save it as a GIfTI image (surface-file representation)
* Save

In [ ]:
nc_file = vis_dir / f'sub-{sub_num}_nc_volume.nii.gz'
pial_lh =     Path(anat_dir / f'sub-{sub_num}_hemi-L_pial.surf.gii')
# inflated_lh = Path(surf_dir / 'lh.inflated')
# white_lh =    Path(anat_dir / f'sub-{sub_num}_hemi-L_white.surf.gii')
#white_other =  Path(surf_dir / 'lh.white')

nc_img = load_img(nc_file)
nc_overlay = surface.vol_to_surf(nc_img, pial_lh) 

img = nib.gifti.GiftiImage(darrays=[nib.gifti.GiftiDataArray(nc_overlay.astype(np.float32))])

# CHANGE THIS TO SAVE INTO SPECIFIC SUBJECT FOLDER and call it by the correct hemisphere, fsnative, and that its derived from white matter surface reference
# nib.save(img, vis_dir / f'sub{sub_num}_nc_pial_lh_fsnative.gii')

In [ ]:
# IGNORE
#inflated_lh_surf = nilearn.surface.load_surf_mesh(inflated_lh)
#pial_lh_surf = nilearn.surface.load_surf_mesh(pial_lh)
#white_lh_surf = nilearn.surface.load_surf_mesh(white_lh)

#print(inflated_lh_surf[0].shape)
#print(pial_lh_surf[0].shape)
#print(white_lh_surf[0].shape)

Notice this is in `fsnative` space, which is subject-specific.

Remember above we imported `fsaverage_infl_left` ? We can see how many vertices are in that mesh and compare it to subject 29 noise ceiling vertex count. 

In [ ]:
fsaverage_vertices, fsaverage_triangles = fsaverage_infl_left
print(fsaverage_vertices.shape)
print(nc_overlay.shape)

The values are different (as expected). If we want to be able to compare multiple subjects, we need to convert subject-specific overlay to one that fits on the `fsaverage` template. You do this by using a Freesurfer program called `mri_surf2surf`. This isn't available in Python and is best run from the terminal directly.

### mri_surf2surf

Need to have a folder with `fsaverage` subject and then the subject-specific Freesurfer folder for each subject (found in `/freesurfer/sourcedata/`). Then need to set environment variable $SUBJECTS_DIR to point to that location so that `mri_surf2surf` knows where to look for the data.


It should look like

    - foldername
      - fsaverage 
      - sub-01
      - sub-02
      - etc ...

And the `sourcedata` freesurfer data should be in each of those folders (not the fMRIprep outputs). Then in terminal set the environment variable SUBJECTS_DIR to `foldername` path by doing the following in the terminal:

`SUBJECTS_DIR=/path/to/foldername`

Then you're ready to run the command below (which you can do individually as not that many subjects or put it in a loop). Just be careful if doing a loop that you're not accidentally cross-loading different subjects data (by updating the source subject ID but not the sval location, for example).

    mri_surf2surf --srcsubject sub-29 \ 
                  --trgsubject fsaverage \
                  --hemi lh \
                  --sval /Users/alxmrphi/Documents/Code/src/sub29_nc_fsnative.gii \ 
                  --tval /Users/alxmrphi/Documents/Data/BirdData/sub-29/misc/sub29_nc_fsaverage.gii


`srcsubject` = source subject (here, subject 29)

`trgsubject` = target subject (here, fsaverage because we want to convert from sub-29 -> fsaverage)

`hemi` = hemisphere (everything is done per-hemisphere with surfaces because they're not modelled as being connected)

`sval` = source value (what file do we want to convert?)

`tval` = target value (what is the filename we want the output to be saved to?)

Once this runs without issue, the new file will be saved into the location specified by `tval`.

### Histogram of high signal ROIs

##### ROI lookup

In [ ]:
roi_lookup = {1: "V1", 2: "MST", 3: "V6", 4: "V2", 5: "V3", 6: "V4", 7: "V8", 8: "4", 9: "3b", 10:	"FEF", 11:	"PEF", 12:	"55b", 13:	"V3A", 14:	"RSC", 15:	"POS2", 16:	"V7", 17:	"IPS1", 18:	"FFC", 19:	"V3B", 20:	"LO1", 21:	"LO2", 22:	"PIT", 23:	"MT", 24:	"A1", 25:	"PSL", 26:	"SFL", 27:	"PCV", 28:	"STV", 29:	"7Pm", 30:	"7m", 31:	"POS1", 32:	"23d", 33:	"v23ab", 34:	"d23ab", 35:	"31pv", 36:	"5m", 37:	"5mv", 38:	"23c", 39:	"5L", 40:	"24dd", 41:	"24dv", 42:	"7AL", 43:	"SCEF", 44:	"6ma", 45:	"7Am", 46:	"7Pl", 47:	"7PC", 48:	"LIPv", 49:	"VIP", 50:	"MIP", 51:	"1", 52:	"2", 53:	"3a", 54:	"6d", 55:	"6mp", 56:	"6v", 57:	"p24pr", 58:	"33pr", 59:	"a24pr", 60:	"p32pr", 61:	"a24", 62:	"d32", 63:	"8BM", 64:	"p32", 65:	"10r", 66:	"47m", 67:	"8Av", 68:	"8Ad", 69:	"9m", 70:	"8BL", 71:	"9p", 72:	"10d", 73:	"8C", 74:	"44", 75:	"45", 76:	"47l", 77:	"a47r", 78:	"6r", 79:	"IFJa", 80:	"IFJp", 81:	"IFSp", 82:	"IFSa", 83:	"p9-46v", 84:	"46", 85:	"a9-46v", 86:	"9-46d", 87:	"9a", 88:	"10v", 89:	"a10p", 90:	"10pp", 91:	"11l", 92:	"13l", 93:	"OFC", 94:	"47s", 95:	"LIPd", 96:	"6a", 97:	"i6-8", 98:	"s6-8", 99:	"43", 100: "OP4", 101: "OP1", 102: "OP2-3", 103: "52", 104: "RI", 105: "PFcm", 106: "PoI2", 107: "TA2", 108: "FOP4", 109: "MI", 110: "Pir", 111: "AVI", 112: "AAIC", 113: "FOP1", 114: "FOP3", 115: "FOP2", 116: "PFt", 117: "AIP", 118: "EC", 119: "PreS", 120: "H", 121: "ProS", 122: "PeEc", 123: "STGa", 124: "PBelt", 125: "A5", 126: "PHA1", 127: "PHA3", 128: "STSda", 129: "STSdp", 130: "STSvp", 131: "TGd", 132: "TE1a", 133: "TE1p", 134: "TE2a", 135: "TF", 136: "TE2p", 137: "PHT", 138: "PH", 139: "TPOJ1", 140: "TPOJ2", 141: "TPOJ3", 142: "DVT", 143: "PGp", 144: "IP2", 145: "IP1", 146: "IP0", 147: "PFop", 148: "PF", 149: "PFm", 150: "PGi", 151: "PGs", 152: "V6A", 153: "VMV1", 154: "VMV3", 155: "PHA2", 156: "V4t", 157: "FST", 158: "V3CD", 159: "LO3", 160: "VMV2", 161: "31pd", 162: "31a", 163: "VVC", 164: "25", 165: "s32", 166: "pOFC", 167: "PoI1", 168: "Ig", 169: "FOP5", 170: "p10p", 171: "p47r", 172: "TGv", 173: "MBelt", 174: "LBelt", 175: "A4", 176: "STSva", 177: "TE1m", 178: "PI", 179: "a32pr", 180: "p24"}

##### Histogram for single participant

In [ ]:
sub_nc_fsaverage = nilearn.surface.load_surf_data(f'E:/fmri_processing/results/visualization/fsaverage_pial/sub{sub_num}_nc_pial_lh_fsaverage.gii')
glasser_annot = nilearn.surface.load_surf_data(Path('E:/fmri_processing/lh.HCPMMP1.annot'))

percent_threshold = 95
threshold = np.percentile(sub_nc_fsaverage, percent_threshold)
mask_array = np.where(sub_nc_fsaverage > threshold, 1, 0)

masked_glasser = glasser_annot * mask_array

# Get unique values and their counts
unique_values, counts = np.unique(masked_glasser[masked_glasser != 0], return_counts=True)

num_rois = 40
top_indices = np.argsort(counts)[-num_rois:]
top_values = unique_values[top_indices]
top_counts = counts[top_indices]

top_values_str = [roi_lookup[val] for val in top_values]
sorted_indices = sorted(range(len(top_values)), key=lambda k: top_values[k])

# Set up a wider plot
fig = plt.figure(figsize=(12, 6))

# Plot the histogram of the top ten values with categorical labels in the original order
plt.bar(np.array(top_values_str)[sorted_indices], np.array(top_counts)[sorted_indices])
plt.xticks(top_values_str, rotation=50)
plt.xlabel('ROIs')
plt.ylabel(f'Per-ROI count')
plt.title(f'Top 40 ROIs within top 5% of noise ceiling values')
plt.show()

##### Histograms for averaged expertise levels

In [ ]:
low_score_subs = [str(val) if val >= 10 else '0' + str(val) for val in [6, 15, 20, 21, 23]]
mod_score_subs = [str(val) if val >= 10 else '0' + str(val) for val in [5, 7, 10, 14, 17, 18, 19, 22]]
high_score_subs = [str(val) if val >= 10 else '0' + str(val) for val in [8, 9, 11, 12, 16, 24]]

low_score_surfs = []
mod_score_surfs = []
high_score_surfs = []

for sub in low_score_subs:
    sub_nc_fsaverage = nilearn.surface.load_surf_data(f'E:/fmri_processing/results/visualization/fsaverage_pial/sub{sub}_nc_pial_lh_fsaverage.gii')
    low_score_surfs.append(sub_nc_fsaverage)
for sub in mod_score_subs:
    sub_nc_fsaverage = nilearn.surface.load_surf_data(f'E:/fmri_processing/results/visualization/fsaverage_pial/sub{sub}_nc_pial_lh_fsaverage.gii')
    mod_score_surfs.append(sub_nc_fsaverage)
for sub in high_score_subs:
    sub_nc_fsaverage = nilearn.surface.load_surf_data(f'E:/fmri_processing/results/visualization/fsaverage_pial/sub{sub}_nc_pial_lh_fsaverage.gii')
    high_score_surfs.append(sub_nc_fsaverage)

low_score_surfs = np.array(low_score_surfs)
mod_score_surfs = np.array(mod_score_surfs)
high_score_surfs = np.array(high_score_surfs)

low_score_surfs = np.mean(low_score_surfs, axis=0)
mod_score_surfs = np.mean(mod_score_surfs, axis=0)
high_score_surfs = np.mean(high_score_surfs, axis=0)

img = nib.gifti.GiftiImage(darrays=[nib.gifti.GiftiDataArray(low_score_surfs.astype(np.float32))])
nib.save(img, f'E:/fmri_processing/results/visualization/low_score_avg_5subs_lh_fsaverage.gii')
img = nib.gifti.GiftiImage(darrays=[nib.gifti.GiftiDataArray(mod_score_surfs.astype(np.float32))])
nib.save(img, f'E:/fmri_processing/results/visualization/mod_score_avg_8subs_lh_fsaverage.gii')
img = nib.gifti.GiftiImage(darrays=[nib.gifti.GiftiDataArray(high_score_surfs.astype(np.float32))])
nib.save(img, f'E:/fmri_processing/results/visualization/high_score_avg_6subs_lh_fsaverage.gii')

levels = ['Low', 'Moderate', 'High']
num_participants = [len(low_score_subs), len(mod_score_subs), len(high_score_subs)]

for i, (surf, level) in enumerate(zip([low_score_surfs, mod_score_surfs, high_score_surfs], levels)):
    glasser_annot = nilearn.surface.load_surf_data(Path('E:/fmri_processing/lh.HCPMMP1.annot'))
    percent_threshold = 95
    threshold = np.percentile(surf, percent_threshold)
    mask_array = np.where(surf > threshold, 1, 0)

    masked_glasser = glasser_annot * mask_array

    # Get unique values and their counts
    unique_values, counts = np.unique(masked_glasser[masked_glasser != 0], return_counts=True)

    num_rois = 25
    top_indices = np.argsort(counts)[-num_rois:]
    top_values = unique_values[top_indices]
    top_counts = counts[top_indices]

    top_values_str = [roi_lookup[val] for val in top_values]
    sorted_indices = sorted(range(len(top_values)), key=lambda k: top_values[k])

    # Set up a wider plot
    fig = plt.figure(figsize=(12, 6))

    # Plot the histogram of the top ten values with categorical labels in the original order
    plt.bar(np.array(top_values_str)[sorted_indices], np.array(top_counts)[sorted_indices])
    plt.xticks(top_values_str, rotation=50)
    plt.xlabel('ROIs')
    plt.ylim(0, 2000)
    plt.ylabel(f'Count of NC Over {percent_threshold}% Threshold in ROI')
    plt.title(f'Top {num_rois} ROIs for the average of {level} Score Participants ({num_participants[i]} ppl)')
    plt.show()

# TASK (over next week)

We want all subject data to be created on `fsnative` and `fsaverage` surfaces and saved in their data folders.

## Can ignore everything below for now, but there if you want to play around.

### Plotting with Python

* Make sure `plotly` is installed for interactive plots (otherwise defaults to static `matplotlib`)
* `!pip install plotly`

In [ ]:
inflated_lh_file = Path(surf_dir / 'lh.inflated')
pial_lh = Path(anat_dir / f'sub-{sub_num}_hemi-L_pial.surf.gii')
white_lh = Path(anat_dir / f'sub-{sub_num}_hemi-L_white.surf.gii')
curve_left_file = surf_dir / 'lh.curv'
curv_left = surface.load_surf_data(curve_left_file)
curv_left_sign = np.sign(curv_left)
sub_nc_fsaverage = nilearn.surface.load_surf_data(vis_dir / f'sub{sub_num}_nc_fsaverage.gii')

In [ ]:
fig = plotting.plot_surf_stat_map(
    inflated_lh_file, nc_overlay, hemi='left',
    bg_map=curv_left_sign,
    threshold=5.5,
    engine='plotly'
)
fig.show()

In [ ]:
fig = plotting.plot_surf_stat_map(
    pial_lh, nc_overlay, hemi='left',
    bg_map=curv_left_sign,
    threshold=5.5,
    engine='plotly'
)
fig.show()

* Going to fsaverage space from fsnative
https://neurostars.org/t/transform-from-fsaverage-to-fsnative/16568

In [ ]:
nib.load()

In [ ]:
sub_nc_fsaverage = nilearn.surface.load_surf_data(vis_dir / f'sub{sub_num}_nc_fsaverage.gii')

In [ ]:
curv_left = surface.load_surf_data(fsaverage.curv_left)
curv_left_sign = np.sign(curv_left)

In [ ]:
from nilearn import plotting

fig = plotting.plot_surf_stat_map(
    fsaverage.infl_right, sub_nc_fsaverage, hemi='left',
    title='Surface left hemisphere', colorbar=True,
    threshold=1., bg_map=curv_left_sign,
)
fig.show()

In [ ]:
from nilearn import plotting

fig = plotting.plot_surf_stat_map(
    fsaverage.infl_left, sub_nc_fsaverage, hemi='left',
    title='Surface left hemisphere', colorbar=True,
    threshold=5.3, bg_map=curv_left_sign, bg_on_data=True,
    engine='plotly'
)
fig.show()

In [ ]:
glasser_lh_path = Path('/Users/alxmrphi/Downloads/lh.HCPMMP1.annot')
glasser = nilearn.surface.load_surf_data(glasser_lh_path)
glasser.shape

In [ ]:
destrieux_atlas = datasets.fetch_atlas_surf_destrieux()
parcellation = destrieux_atlas['map_right']
# these are the regions we want to outline
regions_dict = {b'G_postcentral': 'Postcentral gyrus',
                b'G_precentral': 'Precentral gyrus'}

# get indices in atlas for these labels
regions_indices = [
    np.where(np.array(destrieux_atlas['labels']) == region)[0][0]
    for region in regions_dict
]

labels = list(regions_dict.values())

In [ ]:
np.where(np.array(destrieux_atlas['labels']) == b'G_postcentral')

In [ ]:
V1_idx = 1
V2_idx = 4
V3_idx = 5
V4_idx = 6
V8_idx = 7

labels = ['V1', 'V2', 'V3', 'V4', 'V8']
regions_indices = [1,4,5,6,7]

In [ ]:
from nilearn import plotting

curve_left_file = Path(surf_dir / 'lh.curv')
curv_left = surface.load_surf_data(curve_left_file)
curv_left_sign = np.sign(curv_left)


fig = plotting.plot_surf_stat_map(
    fsaverage.infl_left, sub_nc_fsaverage, hemi='left',
    title='Surface left hemisphere', colorbar=True, view='posterior',
    threshold=5.3, bg_map=curv_left_sign, bg_on_data=True)

plotting.plot_surf_contours(fsaverage.infl_left, glasser, labels=labels,
                            levels=regions_indices, figure=fig,
                            legend=True, engine='plotly')
plotting.show()

In [ ]:
view = plotting.view_surf(fsaverage.infl_left, sub_nc_fsaverage, threshold='98%',
                          bg_map=fsaverage.sulc_left)

view

In [ ]:
view = plotting.view_surf(fsaverage.infl_left, sub_nc_fsaverage, threshold='98%',
                          bg_map=fsaverage.sulc_left)

plotting.plot_surf_contours(fsaverage.infl_left, glasser, labels=labels,
                            levels=regions_indices, figure=view,
                            legend=True, engine='plotly',
                            views=['lateral'])